In [ ]:
import glacierml as gl
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_column',None)
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

In [ ]:
RGI = gl.load_RGI(RGI_path)
glathida = pd.read_csv(os.path.join(glathida_path, 'T.csv'))
glathida = glathida.dropna(subset = ['MEAN_THICKNESS'])
glathida = glathida.reset_index().drop('index',axis = 1)

In [ ]:
if not os.path.exists(
    os.path.join(
        data_path, 'matched.pkl'
)):
    df = gl.match_GlaThiDa_RGI_index(
    RGI,glathida,verbose = True, useMP = True
)
    
    df.to_pickle(os.path.join(data_path,'matched.pkl'))
    
else:
    print('data already matched')